In [5]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io as sio
import mne
import pandas as pd

In [6]:
# Function For Calculating PSD from a trial such as "jl_eeg3" 
# Return a 1D array of all the channel's Band's PSD

def calculate_psd_features(eeg_trial_data):
    # Create MNE Raw object
    info = mne.create_info(ch_names=[f'Channel {i + 1}' for i in range(eeg_trial_data.shape[0])], sfreq=200, ch_types='eeg')
    raw = mne.io.RawArray(eeg_trial_data, info)

    # Define frequency bands
    freq_bands = {'delta': (1, 4), 'theta': (4, 8), 'alpha': (8, 14), 'beta': (14, 30), 'gamma': (30, 50)}

    # Calculate PSD Features using Welch method
    psds, freqs = mne.time_frequency.psd_array_welch(raw.get_data(), sfreq=raw.info['sfreq'], fmin=0.5, fmax=50, n_jobs=1)

    # Organize PSD Features into DataFrame
    psd_data = []

    # Iterate through frequency bands
    for band, (fmin, fmax) in freq_bands.items():
        band_data = []

        # Iterate through channels
        for channel_idx, channel_name in enumerate(info.ch_names):
            freq_mask = (freqs >= fmin) & (freqs <= fmax)
            avg_psd = np.mean(psds[channel_idx, freq_mask])
            band_data.append(avg_psd)

        # Append channel's PSD features to the data list
        psd_data.extend(band_data)

    return psd_data

# Example usage:
# psd_data = calculate_psd_features(eeg_trial_data)
# print(psd_data)


In [9]:
# Calculate all the mat file's all trial's PSD 
# Return DF containing trial number and each trial's all channel's PSD

def process_subject_mat_files(mat_folder):
    freq_bands = {'delta': (1, 4), 'theta': (4, 8), 'alpha': (8, 14), 'beta': (14, 30), 'gamma': (30, 50)}
    subject_psd_data = []

    # Iterate through mat files in the folder
    for filename in os.listdir(mat_folder):
        if filename.endswith(".mat"):
            mat_file = os.path.join(mat_folder, filename)

            # Load the mat file
            mat_content = sio.loadmat(mat_file)

            # Identify trials in the mat file
            trial_names = [key for key in mat_content.keys() if 'eeg' in key.lower()]
            trial_names.sort()
            # Iterate through trials
            for trial_name in trial_names:
                eeg_trial_data = mat_content[trial_name]
                psd_features = calculate_psd_features(eeg_trial_data)
                subject_psd_data.append(psd_features)
    
    column_names = [f'channel_{i}_{band}' for i in range(62) for band in freq_bands.keys()]
    df = pd.DataFrame(data=np.array(subject_psd_data), columns=column_names)

    
    return df



In [10]:
mat_folder = "E:/STUDY/Publications/Thesis/Brain Emotion Detection/Dataset/Practice with datasets/Seed/Preprocessed_EEG"
subject_psd_data = process_subject_mat_files(mat_folder)

Creating RawArray with float64 data, n_channels=62, n_times=47001
    Range : 0 ... 47000 =      0.000 ...   235.000 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=62, n_times=47401
    Range : 0 ... 47400 =      0.000 ...   237.000 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=62, n_times=47001
    Range : 0 ... 47000 =      0.000 ...   235.000 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=62, n_times=46601
    Range : 0 ... 46600 =      0.000 ...   233.000 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=62, n_times=47001
    Range : 0 ... 47000 =      0.000 ...   235.000 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with float64 data, n_channels=62, n_times=47601
    Range : 0 ... 47600 =      0.000 ...   238.000 secs
Ready.
Effective window size : 1.280 (s)
Creating RawArray with

In [11]:
subject_psd_data.shape

(30, 310)

In [12]:
subject_psd_data.head()

,channel_0_delta,channel_0_theta,channel_0_alpha,channel_0_beta,channel_0_gamma,channel_1_delta,channel_1_theta,channel_1_alpha,channel_1_beta,channel_1_gamma,...,channel_60_delta,channel_60_theta,channel_60_alpha,channel_60_beta,channel_60_gamma,channel_61_delta,channel_61_theta,channel_61_alpha,channel_61_beta,channel_61_gamma
0,151.761753,140.103556,135.821030,57.546267,48.765136,105.625168,37.469691,20.662526,14.989260,12.151366,...,0.148085,0.089088,0.122542,12929.385864,0.523258,0.710669,0.566447,0.268231,0.214262,0.195238
1,221.985420,207.570899,201.253253,85.006936,70.076321,376.459245,57.575279,30.790129,21.831946,17.489631,...,0.179611,0.099016,0.154036,99411.811384,0.556977,1.161855,0.827118,0.334716,0.311854,0.289642
2,150.244008,144.525913,145.000126,57.608473,51.504297,318.260798,40.100752,21.109848,15.158482,12.995876,...,0.113547,0.074127,0.107916,99389.451914,0.326249,0.609443,0.449566,0.244761,0.247501,0.248170
3,161.931320,151.890108,144.587309,62.905725,50.289912,564.938347,43.470552,22.892416,16.665629,13.651681,...,0.113309,0.066728,0.088432,96822.736966,0.333633,0.617128,0.456924,0.224101,0.157413,0.149834
4,130.603259,122.207883,117.894659,50.354839,42.269319,469.924648,37.862057,18.132463,13.439139,11.102117,...,0.139576,0.078573,0.101031,130396.274570,0.393610,0.890892,0.620245,0.276692,0.185274,0.178883
